본문은 권창현 교수님의 줄리아 책, julia programming for Operations Rsearch를 참고하여 정리하였습니다.
> https://www.softcover.io/read/7b8eb7d0/juliabook2/frontmatter


# Chapter 2. Simple Linear Optimization

## 2.1 Linear programming (LP) Problems


$$max \sum_{i=1}^3 c_i x_i$$

subject to

$$\mathbf{Ax \le b}$$

=
$$-x_1 + x_2 +3x_3 \le -5$$
$$x_1 + 3x_2 -7x_3 \le 10$$
$$0\le x_i, i = \{1,2,3\} $$
$$x_1\le10$$


In [2]:
using JuMP, Clp

#=
모델을 선언해준다. 여기서는 Clp를 사용했다.
=#
m = Model(with_optimizer(Clp.Optimizer))


#=
변수를 선언해준다.
@variable은 JuMP 패키지에서 macros라고 하며, 이는 작업을 반복적으로 해준다.
@objective도 함께 해준다!
=#
@variable(m, 0<= x1 <=10)
@variable(m, x2 >=0)
@variable(m, x3 >=0)

@objective(m, Max, x1 + 2x2 + 5x3)

#=
'constraint1, 2'는 제약조건의 이름이며, 이는 추후 dual variable 값들과 연관된 것을 찾을때 유용하다.
=#
@constraint(m, constraint1, -x1 + x2 + 3x3 <= -5)
@constraint(m, constraint2, x1 + 3x2 - 7x3 <= 10)

#=
모델을 확인해준다.
=#
print(m)

Max x1 + 2 x2 + 5 x3
Subject to
 constraint1 : -x1 + x2 + 3 x3 ≤ -5.0
 constraint2 : x1 + 3 x2 - 7 x3 ≤ 10.0
 x1 ≥ 0.0
 x2 ≥ 0.0
 x3 ≥ 0.0
 x1 ≤ 10.0


┌ Warning: `with_optimizer` is deprecated. Adapt the following example to update your code:
│ `with_optimizer(Ipopt.Optimizer)` becomes `Ipopt.Optimizer`.
│   caller = top-level scope at In[2]:2
└ @ Core In[2]:2


In [3]:
#=
솔버를 작동!
=#
JuMP.optimize!(m)

Coin0506I Presolve 2 (0) rows, 3 (0) columns and 6 (0) elements
Clp0006I 0  Obj 0 Primal inf 1.6666666 (1) Dual inf 12.666666 (3)
Clp0006I 2  Obj 19.0625
Clp0000I Optimal - objective value 19.0625
Clp0032I Optimal objective 19.0625 - 2 iterations time 0.002


In [4]:
#=
JuMP.value()에 결과값들이 있다. 
여기서 println()은 결과값을 프린트하고 한 줄을 띈다. 그렇게 하고 싶지 않다면 print()를 사용하자.
=#
println("Optimal solutions:")
println("x1 = ", JuMP.value(x1))
println("x2 = ", JuMP.value(x2))
println("x3 = ", JuMP.value(x3))

#=
optimal dual variables 값들을 얻기 위해 JuMP.shadow_price()를 제약조건 이름과 함께 사용한다.
=#
println("Dual Variables:")
println("dual1 = ", JuMP.shadow_price(constraint1))
println("dual2 = ", JuMP.shadow_price(constraint2))

#=
JuMP.dual()이라는 함수도 정의되어있다. 하지만 이 사인은 기대했던 결과가 아닌데, conic duality의 규칙을 따르기 때문이다.
선형 최적화 문제의 경우 JuMP.shadow_price()가 대부분의 표준 교과서에와 같은 값을 제공한다.
=#
println("Dual Variables (conic):")
println("dual1 = ", JuMP.dual(constraint1))
println("dual2 = ", JuMP.dual(constraint2))

Optimal solutions:
x1 = 10.0
x2 = 2.1875
x3 = 0.9374999999999999
Dual Variables:
dual1 = 1.8125
dual2 = 0.0625
Dual Variables (conic):
dual1 = -1.8125
dual2 = -0.0625


## 2.2 Alternative Ways of Writing LP Problems

위 예제는 array 로 변수를 정했다. 하지만 다음과 같이 vector로도 할수 있다.

In [11]:
m = Model(with_optimizer(Clp.Optimizer))

@variable(m, x[1:3] >= 0)

c = [1; 2; 5]
@objective(m, Max, sum( c[i]*x[i] for i in 1:3))

A =[-1 1 3; 1 3 -7] #제약조건들의 계수
b = [-5; 10] #rhs

@constraint(m, constraint1, sum( A[1, i] * x[i] for i in 1:3) <= b[1])
@constraint(m, constraint2, sum( A[2, i] * x[i] for i in 1:3) <= b[2])
@constraint(m, bound, x[1] <= 10)

JuMP.optimize!(m)

┌ Warning: `with_optimizer` is deprecated. Adapt the following example to update your code:
│ `with_optimizer(Ipopt.Optimizer)` becomes `Ipopt.Optimizer`.
│   caller = top-level scope at In[11]:1
└ @ Core In[11]:1


Coin0506I Presolve 2 (-1) rows, 3 (0) columns and 6 (-1) elements
Clp0006I 0  Obj 4.8999999 Primal inf 0.033332367 (1) Dual inf 12.666664 (3)
Clp0006I 2  Obj 19.0625
Clp0000I Optimal - objective value 19.0625
Coin0511I After Postsolve, objective 19.0625, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 19.0625 - 2 iterations time 0.002, Presolve 0.00


In [12]:
println("Optimal solutions:")
println("x1 = ", JuMP.value(x1))
println("x2 = ", JuMP.value(x2))
println("x3 = ", JuMP.value(x3))

println("Dual Variables:")
println("dual1 = ", JuMP.shadow_price(constraint1))
println("dual2 = ", JuMP.shadow_price(constraint2))

Optimal solutions:
x1 = 10.0
x2 = 2.1875
x3 = 0.9374999999999999
Dual Variables:
dual1 = 1.8125
dual2 = 0.0625


제약조건이 많아지면 다음과 같이 Dictionary를 사용하는 게 더 효과적이다.

In [16]:
m = Model(with_optimizer(Clp.Optimizer))

c = [ 1; 2; 5]
A = [-1  1  3;
      1  3 -7]
b = [-5; 10]

index_x = 1:3
index_constraints = 1:2

@variable(m, x[index_x] >= 0)
@objective(m, Max, sum( c[i]*x[i] for i in index_x) )

@constraint(m, constraint[j in index_constraints],
               sum( A[j,i]*x[i] for i in index_x ) <= b[j] )
@constraint(m, bound, x[1] <= 10)

JuMP.optimize!(m)

┌ Warning: `with_optimizer` is deprecated. Adapt the following example to update your code:
│ `with_optimizer(Ipopt.Optimizer)` becomes `Ipopt.Optimizer`.
│   caller = top-level scope at In[16]:1
└ @ Core In[16]:1


Coin0506I Presolve 2 (-1) rows, 3 (0) columns and 6 (-1) elements
Clp0006I 0  Obj 4.8999999 Primal inf 0.033332367 (1) Dual inf 12.666664 (3)
Clp0006I 2  Obj 19.0625
Clp0000I Optimal - objective value 19.0625
Coin0511I After Postsolve, objective 19.0625, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 19.0625 - 2 iterations time 0.002, Presolve 0.00


In [17]:
println("Optimal Solutions:")
for i in 1:3
  println("x[$i] = ", JuMP.value(x[i]))
end

println("Dual Variables:")
for j in 1:2
  println("dual[$j] = ", JuMP.shadow_price(constraint[j]))
end

Optimal Solutions:
x[1] = 10.0
x[2] = 2.1875
x[3] = 0.9374999999999999
Dual Variables:
dual[1] = 1.8125
dual[2] = 0.0625


In [109]:
print(t[3])

Int

## 2.4 Mixed Inteer Linear Programming (MILP) Problems

$$ \max x_1 + 2x_2 + 5x_3$$
subject to

$$
\begin{matrix}
-x_1+x_2+3x_3 &\le& -5 \\
x_1 +3x_2 -7x_3 &\le&  10 \\
0 \le x_1 &\le& 10 \\
x_2 &\ge& Integer \\
x_3 &\in& \{0, 1\}
\end{matrix}
$$

In [116]:
using MathOptInterface
const MOI = MathOptInterface

m = Model(with_optimizer(Clp.Optimizer))

c = [ 1; 2; 5]
A = [-1  1  3;
      1  3 -7]
b = [-5; 10]
t = [ :Bin; :Bin; :Int]
lb = [0;0;0]
ub = [Inf;Inf;10]
index_x = 1:3
index_constraints = 1:2

```
for i in index_x
    println(i)
    if t[i] == ":Bin"
        @variable(m, x[i], Bin)
    elseif t[i] == ":Int"
        @variable(m, x[i], Int)
    else
        @variable(m, x[i])
    end
end
```

@variable(m, x[i in index_x])

for i in index_x
    println(t[i])
    if t[i] == :Bin
        @constraint(m, x[i] in MOI.ZeroOne())
    else t[i] == :Int
        @constraint(m, x[i] in MOI.Integer())
    end
end

    #@variable(m, lb[i] <= x[i] <= ub[i] for i in index_x)
@objective(m, Max, sum( c[i]*x[i] for i in index_x) )
@constraint(m, constraint[j in index_constraints],
               sum( A[j,i]*x[i] for i in index_x ) <= b[j] )

@constraint(m, bound[j in index_x],
               lb[j]<= x[j] <= ub[j]  )

print(m)

┌ Warning: `with_optimizer` is deprecated. Adapt the following example to update your code:
│ `with_optimizer(Ipopt.Optimizer)` becomes `Ipopt.Optimizer`.
│   caller = top-level scope at In[116]:3
└ @ Core In[116]:3


Bin


ErrorException: Constraints of type MathOptInterface.SingleVariable-in-MathOptInterface.ZeroOne are not supported by the solver and there are no bridges that can reformulate it into supported constraints.

In [110]:
JuMP.optimize!(m)

# Printing the optimal solutions obtained
println("Optimal Solutions:")
println("x1 = ", JuMP.value(x1))
println("x2 = ", JuMP.value(x2))
println("x3 = ", JuMP.value(x3))

MathOptInterface.UnsupportedConstraint{MathOptInterface.SingleVariable,MathOptInterface.ZeroOne}: MathOptInterface.UnsupportedConstraint{MathOptInterface.SingleVariable,MathOptInterface.ZeroOne}: `MathOptInterface.SingleVariable`-in-`MathOptInterface.ZeroOne` constraint is not supported by the model.

In [114]:
print(t[3])

Int

In [53]:
@constraint(m, bound[i], x[i] <= ub[i])
for i in index_x
        print(i)
        @constraint(m, bound[i], x[i] <= ub[i])
end


1

ErrorException: An object of name bound is already attached to this model. If this is intended, consider using the anonymous construction syntax, e.g., x = @variable(model, [1:N], ...) where the name of the object does not appear inside the macro.

In [ ]:
@objective(m, Max, sum( c[i]*x[i] for i in index_x) )

@constraint(m, constraint[j in index_constraints],
               sum( A[j,i]*x[i] for i in index_x ) <= b[j] )
@constraint(m, bound, x[1] <= 10)


@variable(m, 0<= x1 <=10)
@variable(m, x2 >=0, Int)
@variable(m, x3, Bin)